In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

In [ ]:
from IPython.display import display, Javascript

display(Javascript('''
(function() {
    var interval = setInterval(function() {
        if (typeof google !== 'undefined' && google.translate && google.translate.TranslateElement) {
            clearInterval(interval);
            google.translate.TranslateElement = function() {};
            document.getElementById('google_translate_element')?.remove();
        }
    }, 1000);
})();
'''))

## Update repository

In [ ]:
! git pull

## Add import path

In [ ]:
import os
import sys
import gc

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
del module_path

## Organize imports

In [ ]:
import multiprocessing
from pathlib import Path

In [ ]:
from collections import OrderedDict

In [ ]:
from functools import reduce

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from tqdm import tqdm

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import FashionMNIST

In [ ]:
from src.lattmc.fca.utils import *
from src.lattmc.fca.data_utils import *
from src.lattmc.fca.image_utils import *
from src.lattmc.fca.models import *
from src.lattmc.fca.fca_utils import *
from src.lattmc.fca.image_gens import *

#### Number of CPU cores

In [ ]:
workers = multiprocessing.cpu_count()
workers

In [ ]:
SEED = 2024

## Initialize Path

In [ ]:
PATH = Path('data')
model_dir = PATH / 'models'
model_path = model_dir / 'simple_cnn_fasion_mnist_model.ckpt'
config_dir = PATH / 'config'
config_dir.mkdir(exist_ok=True, parents=True)
config_1_layer_path = config_dir / 'neurons_1_layer.json'
config_cnn_layer_1 = config_dir / 'neurons_cnn_1_layer.json'
images_dir = PATH / 'images'
images_dir.mkdir(exist_ok=True, parents=True)
pumpkin_path = PATH / 'Pumpkin_Seeds_Dataset.xlsx'

## Load the model

In [ ]:
model = torch.load(model_path, map_location='cpu')

In [ ]:
def clear_state_dict(state_dict):
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.0.', 'conv1.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.3.', 'conv2.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.8.', 'fc1.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.11.', 'fc2.')] = state_dict.pop(key)

    return state_dict

In [ ]:
state_dict = clear_state_dict(model['state_dict'])

In [ ]:
net = nn.Sequential(OrderedDict([
    ('conv1', nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)),
    ('act1', nn.ReLU()),
    ('mxp1', nn.MaxPool2d(kernel_size=2, stride=2)),
    ('conv2', nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)),
    ('act2', nn.ReLU()),
    ('mxp2', nn.MaxPool2d(kernel_size=2, stride=2)),
    ('flatten', nn.Flatten()),
    ('fc1', nn.Linear(64 * 7 * 7, 128)),
    ('act3', nn.ReLU()),
    ('fc2', nn.Linear(128, 10)),
]))

In [ ]:
net.load_state_dict(state_dict)

In [ ]:
net = net.eval()

## Initialize FashionMNIST dataset

In [ ]:
mean, std = compute_mean_std(
    FashionMNIST(
        images_dir, 
        train=True, 
        download=True, 
        transform=transforms.Compose(
            [
                transforms.ToTensor(),
              ]
            )
        ),
    workers=workers
    )

In [ ]:
mean, std

In [ ]:
transform = transforms.Compose(
            [
                ToTensor(),
                transforms.Normalize((mean,), (std,)),
            ]
)

In [ ]:
data_train = FashionMNIST(images_dir, train=True, download=True)
data_test = FashionMNIST(images_dir, train=False, download=True)

In [ ]:
next(net.parameters()).device

In [ ]:
device = find_device()
device

In [ ]:
wnet = NetWrapper(net, transform)

In [ ]:
wnet.net

In [ ]:
wnet.net[:6]

In [ ]:
layer_V_n = 3
layer_U_n = 6

In [ ]:
wnet.device

In [ ]:
bs = 8

In [ ]:
V_X_train, X_V_train = layer_V(data_train, wnet, k=layer_V_n, bs=bs)

In [ ]:
V_X_test, X_V_test = layer_V(data_test, wnet, k=layer_V_n, bs=bs)

In [ ]:
U_X_train, X_U_train = layer_V(data_train, wnet, k=layer_U_n, bs=bs)

In [ ]:
U_X_test, X_U_test = layer_V(data_test, wnet, k=layer_U_n, bs=bs)

In [ ]:
V_X_train.shape, V_X_test.shape, U_X_train.shape, U_X_test.shape

## Sorting vectors

In [ ]:
V_X_digits, V_X_sorteds = sort_V_X(V_X_train, data_train)

## Alanyze maximum stimulus

In [ ]:
v_Ds = dict()
u_Ds = dict()
G_v_tests = dict()
G_u_tests = dict()
uncn_dict = dict()

In [ ]:
i = 0
ths = [
    328, #0
    280, #1
    320, #2
    384, #3
    300, #4
    300, #5
    400, #6
    200, #7
    380, #8
    180  #9
]
v = np.copy(V_X_sorteds[i][ths[i]])

In [ ]:
for i in range(10):
    layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
    G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
        v, 
        V_X_test, 
        U_X_test, 
        data_test
    )
    v_Ds[i] = v_D
    u_Ds[i] = u_D
    G_v_tests[i] = G_v_test
    G_u_tests[i] = G_u_test
    uncn_dict[i] = uncn_reps

In [ ]:
uncn_dict[i]

In [ ]:
show_grid(G_v_tests[i], data_test, nrow=32)

In [ ]:
show_grid(G_u_tests[i], data_test, nrow=32)

In [ ]:
y_hs = [np.argmax(wnet(data_test[idx][0])) for idx in G_u_test]

In [ ]:
uncn_hat = layer_fca.count_ys(y_hs)

In [ ]:
show_grid(G_u_tests[i], data_test, nrow=48, h=64, w=64, my=i)

## Experiments with shapes

In [ ]:
# Example usage
dataset = CustomShapeDataset(num_samples=1000)
# dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

shapes_shp = get_digits(dataset)

# Get a batch of images
images, labels = zip(*[(x, y) for idx_l, (x, y) in enumerate(dataset) if idx_l < 32])

# Show a batch of images
show_images(images[:16], labels[:16])

In [ ]:
ring = np.copy(shapes_shp['nring'][0][0].numpy())
ring[-13 :, :] = 0
plt.imshow(ring)

In [ ]:
v_X_shapes_c = wnet(ring, k=layer_V_n)
visualize_slices(v_X_shapes_c[0])

In [ ]:
max_index, max_val = argmax_kd_val(v_X_shapes_c[0])
max_index, max_index[0], max_val

In [ ]:
c_idx = 0
max_index, max_val = argmax_kd_val(v_X_shapes_c[c_idx])
print(max_index, max_index[0], max_val)
fl_c = max_index[0]
neurons_c = np.zeros(v_X_shapes_c[c_idx].shape)
th = max_val - max_val / 4.2
idxs_c = np.where(v_X_shapes_c[c_idx][fl_c] >= th)
idx_c_c = idxs_c[1]
val_c = 1.0
# idx_d_c -= 1
neurons_c[fl_c][idxs_c] = val_c
# neurons_c[fl_c][4, 7] = val_c
# neurons_c[fl_c][5, 6] = val_c
# neurons_c[fl_c][5, 8] = val_c
# neurons_c[fl_c][6, 6] = 0
# neurons_c[fl_c][6, 7] = 0
# neurons_c[fl_c][6, 8] = 0
# neurons_d[fl_c][9:14, idx_d_c[0]] = 0
v_c = np.copy(neurons_c)
show_activation(neurons_c[fl_c])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_c, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
v_X_shapes_v = np.array(wnet(*shapes_shp['vertical_line'], k=layer_V_n))
v_X_shapes_h = np.array(wnet(*shapes_shp['horizontal_line'], k=layer_V_n))
v_X_shapes_e = np.array(wnet(*shapes_shp['sring'], k=layer_V_n))
v_X_shapes_r = np.array(wnet(*shapes_shp['nring'], k=layer_V_n))
v_X_shapes_v.shape, v_X_shapes_h.shape, v_X_shapes_e.shape, v_X_shapes_r.shape

In [ ]:
i = 3

In [ ]:
show_img(shapes_shp['vertical_line'], i)

In [ ]:
visualize_slices(v_X_shapes_h[i])

In [ ]:
diag = torch.zeros((28, 28), dtype=torch.float32)
for i in range(8, 24):
    diag[28 - i, i] = 255
diag /= 255
# diag = diag.t()
plt.imshow(diag)

In [ ]:
v_X_shapes_d = wnet(diag.unsqueeze(0), k=layer_V_n)

In [ ]:
visualize_slices(v_X_shapes_d[0])

In [ ]:
d_idx = 0
max_index, max_val = argmax_kd_val(v_X_shapes_d[d_idx])
print(max_index, max_index[0], max_val)
fl_d = max_index[0]
neurons_d = np.zeros(v_X_shapes_d[d_idx].shape)
th = max_val - max_val / 4
idxs_d = np.where(v_X_shapes_d[d_idx][fl_d] >= th)
idx_d_c = idxs_d[1]
# idx_d_c -= 1
neurons_d[fl_d][idxs_d] = 0.2
# neurons_d[fl_d][2:8, idx_d_c[0]] = 0
# neurons_d[fl_d][9:14, idx_d_c[0]] = 0
v_d = np.copy(neurons_d)
show_activation(neurons_d[fl_d])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_d, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=64)

In [ ]:
visualize_slices(v_X_shapes_v[0])

In [ ]:
v_idx = 0
max_index, max_val = argmax_kd_val(v_X_shapes_v[v_idx])
print(max_index, max_index[0], max_val)
fl_v= max_index[0]
nz_idx_v = gen_line_idx(hv_shift=8, sid=3, eid=12, hv='v')
neurons_v = np.zeros(v_X_shapes_d[v_idx].shape)
neurons_v[fl_v][nz_idx_v] = 0.2
v_v = np.copy(neurons_v)
show_activation(neurons_v[fl_v])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_v, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
visualize_slices(v_X_shapes_h[0])

In [ ]:
h_idx = 0
max_index, max_val = argmax_kd_val(v_X_shapes_h[h_idx])
print(max_index, max_index[0], max_val)
fl_h = max_index[0]
nz_idx_h = gen_line_idx(hv_shift=3, sid=4, eid=9, hv='h')
neurons_h = np.zeros(v_X_shapes_h[h_idx].shape)
neurons_h[fl_h][nz_idx_h] = 0.2
v_h = np.copy(neurons_h)
show_activation(neurons_h[fl_h])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_h, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
v_l = [v_v, v_h]
v = np.max(np.array(v_l), axis=0)
v.shape

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
visualize_slices(v_X_shapes_r[i])

In [ ]:
r_idx = 0
max_index, max_val = argmax_kd_val(v_X_shapes_r[r_idx])
print(max_index, max_index[0], max_val)
fl_r = max_index[0]
neurons_r = np.zeros(v_X_shapes_r[r_idx].shape)
th = max_val - max_val / 4
idxs_r = np.where(v_X_shapes_r[r_idx][fl_r] >= th)
idx_r_c = idxs_r[1]
# idx_d_c -= 1
neurons_r[fl_r][idxs_r] = 0.1
# neurons_d[fl_d][2:8, idx_d_c[0]] = 0
# neurons_d[fl_d][9:14, idx_d_c[0]] = 0
v_r = np.copy(neurons_r)
show_activation(neurons_r[fl_r])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_r, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
v_l = [
    # v_v, 
    v_h, 
    v_r, 
    v_d
]
v = np.max(np.array(v_l), axis=0)
v.shape

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=64)